# Primary Care Performance Analysis
This notebook analyzes a mock healthcare dataset (`data.csv`) to produce provider-level KPIs and visualizations.

Files in this repo:
- `data.csv` — mock dataset
- `analysis.ipynb` — this notebook
- `provider_summary.csv` — aggregated KPI results
- `dashboard.png` — visualization snapshot


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('data.csv')
df.head()


In [ ]:
# Provider attribution
provider_counts = df.groupby('Provider_Name')['Patient_ID'].nunique().sort_values(ascending=False)
provider_counts

In [ ]:
# Calculate total visits as well
total_visits = df.groupby("Provider_Name").size().reset_index(name="TotalVisits")
# Create a DataFrame for provider counts
provider_counts = provider_counts.reset_index(name="UniquePatients")
# Rename columns for clarity
provider_counts.columns = ['Provider_Name', 'UniquePatients']
# Plotting the provider counts
plt.figure(figsize=(10, 6))
plt.bar(provider_counts['Provider_Name'], provider_counts['UniquePatients'], color='skyblue')
plt.xlabel('Provider Name')
plt.ylabel('Unique Patients')
plt.title('Unique Patients per Provider')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Save the provider counts to a CSV file
provider_counts.to_csv("provider_counts.csv", index=False)
# Save the total visits to a CSV file
total_visits.to_csv("total_visits.csv", index=False)

# Combine provider counts and total visits into a summary DataFrame
provider_summary = pd.DataFrame({
    'Provider_Name': provider_counts['Provider_Name'],
    'UniquePatients': provider_counts['UniquePatients'],
    'TotalVisits': total_visits['TotalVisits']
})
# Save the summary DataFrame to a CSV file
provider_summary.to_csv("provider_summary.csv", index=False)

In [ ]:
# Quality metric compliance
quality_rate = df.groupby('Provider_Name')['Quality_Metric'].apply(lambda x: (x=='Met').mean()*100)
# Create a DataFrame for quality rates
quality_rate_df = quality_rate.reset_index(name='QualityRate')
# Rename columns for clarity
quality_rate_df.columns = ['Provider_Name', 'QualityRate']
# Plotting the quality rates
plt.figure(figsize=(10, 6))
plt.bar(quality_rate_df['Provider_Name'], quality_rate_df['QualityRate'], color='lightgreen')
plt.xlabel('Provider Name')
plt.ylabel('Quality Rate (%)')
plt.title('Quality Metric Compliance per Provider')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
# Save the quality rate DataFrame to a CSV file
quality_rate_df.to_csv("quality_rate.csv", index=False)

# Debug Code
#print("Unique values in Quality_Metric:")
#print(df['Quality_Metric'].value_counts(dropna=False))
#print("\nFirst 10 values:")
#print(df['Quality_Metric'].head(10))
#print("\nData type:")
#print(df['Quality_Metric'].dtype)


In [ ]:
#Debug Cell
print("provider_counts type:", type(provider_counts))
print("provider_counts shape:", provider_counts.shape)
print("provider_counts columns:", provider_counts.columns.tolist() if hasattr(provider_counts, 'columns') else 'No columns (it\'s a Series)')
print("provider_counts index:", provider_counts.index.tolist()[:10])  # First 10
print("provider_counts head:")
print(provider_counts.head())

print("total_visits type:", type(total_visits))
print("total_visits shape:", total_visits.shape)
print("total_visits index:", total_visits.index.tolist())
print("total_visits head:")
print(total_visits.head())

In [ ]:
# Save summary
provider_df = provider_counts.reset_index()  # Convert Series to DataFrame
provider_df.columns = ['Provider_Name', 'UniquePatients']

# Merge with total_visits to align properly
merged = provider_df.merge(total_visits, on='Provider_Name', how='left')

# Then add quality rate
summary = merged.merge(quality_rate_df, on='Provider_Name', how='left')

print(summary)


In [ ]:
# Plot (creates dashboard.png)
fig, axes = plt.subplots(2,2, figsize=(12,10))
plt.tight_layout(pad=4.0)
axes[0,0].bar(provider_counts.index, provider_counts.values)
axes[0,0].set_title('Patients per Provider')
axes[0,0].set_ylabel('Unique Patients')
axes[0,0].tick_params(axis='x', rotation=45, labelsize=8)
axes[0,1].bar(quality_rate.index, quality_rate.values)
axes[0,1].set_title('Quality Metric Compliance (%)')
axes[0,1].set_ylabel('Compliance %')
axes[0,1].tick_params(axis='x', rotation=45, labelsize=8)
visit_counts = df['Visit_Type'].value_counts()
axes[1,0].pie(visit_counts.values, labels=visit_counts.index, autopct='%1.1f%%')
axes[1,0].set_title('Visit Type Distribution')
top_diag = df['Diagnosis_Code'].value_counts().head(5)
axes[1,1].bar(top_diag.index, top_diag.values)
axes[1,1].set_title('Top 5 Diagnoses')
fig.savefig('dashboard.png', bbox_inches='tight', dpi=150)
plt.show()


In [ ]:
## Methods and Limitations

** Methods **
- **Data Generation**: This dataset is synthetic and was created for demonstration purposes. Patient IDs, Provider IDs, and visit data were randomly generated to simulate realistic patterns without containing any real patient information.
- ** KPIs Computed:**
- **Visits per Provider:** Total count of patient visits grouped by provider.  
  - **Compliance Rate:** Percentage of visits marked as compliant (quality metric met).  
  - **Average Wait Time:** Mean time between scheduled and actual appointment.
  - **Provider Attribution** Paitents per Provider was calculated as the unique number of                                     patients assigned to each provider.
- **Tools Used:** Python (Pandas for data wrangling, Matplotlib/Seaborn for visualization).
- An interactive HTML Dashboard was built using plotly to display provider performance metrics, patient volume, and quality compliance rates.

**Limitations**
- **No Real Patient Data:** These results do not reflect actual clinical performance and are for portfolio purposes only.
- **Simplified Metrics:** Compliance is represented as a binary field without accounting for measure-specific thresholds or exclusions.
- **No Seasonality or Time Trends:** Data generation did not include real-world variations such as seasonal illness spikes or policy changes.